In [8]:
data = spark.read.csv("file:/home/hduser/insurance.csv", inferSchema = True, header = True)

In [10]:
data.show()

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|
| 60|female| 25.84|       0|    no|northwest|28923.13692|
| 25|  male| 26.22|       0|    no|northeast|  2721.3208|
| 62|female| 26.29|       0|   yes|southeast| 27808.7251|
| 23|  male|  34.4|       0|    no|southwest|   1826.843|
| 56|female| 39.82|       0|    no|southeast| 11090.7178|
| 27|  male| 4

In [12]:
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)



In [14]:
data.describe().show()

+-------+------------------+------+------------------+-----------------+------+---------+------------------+
|summary|               age|   sex|               bmi|         children|smoker|   region|           charges|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+
|  count|              1338|  1338|              1338|             1338|  1338|     1338|              1338|
|   mean| 39.20702541106129|  null|30.663396860986538|  1.0949177877429|  null|     null|13270.422265141257|
| stddev|14.049960379216147|  null| 6.098186911679012|1.205492739781914|  null|     null|12110.011236693992|
|    min|                18|female|             15.96|                0|    no|northeast|         1121.8739|
|    max|                64|  male|             53.13|                5|   yes|southwest|       63770.42801|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+



In [16]:
age = data.corr("age", "charges")
BMI = data.corr("bmi", "charges")

print("Correlation between Age of the person and charges is : {}".format(age))
print("Correlation between BMI of the person and charges is : {}".format(BMI))

Correlation between Age of the person and charges is : 0.299008193330648
Correlation between BMI of the person and charges is : 0.19834096883362903


In [18]:
from pyspark.ml.feature import StringIndexer
category = StringIndexer(inputCol = "sex", outputCol = "gender_categorical")
categorised = category.fit(data).transform(data)
categorised.show()


category = StringIndexer(inputCol = "smoker", outputCol = "smoker_categorical")
categorised = category.fit(categorised).transform(categorised)
categorised.show()

category = StringIndexer(inputCol = "region", outputCol = "region_categorical")
categorised = category.fit(categorised).transform(categorised)
categorised.show()

+---+------+------+--------+------+---------+-----------+------------------+
|age|   sex|   bmi|children|smoker|   region|    charges|gender_categorical|
+---+------+------+--------+------+---------+-----------+------------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|               1.0|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|               0.0|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|               0.0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|               0.0|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|               0.0|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|               1.0|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|               1.0|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|               1.0|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|               0.0|
| 60|female| 25.84|       0|    no|northwest|28923.13692|               1.0|

In [20]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
categorised.columns

['age',
 'sex',
 'bmi',
 'children',
 'smoker',
 'region',
 'charges',
 'gender_categorical',
 'smoker_categorical',
 'region_categorical']

In [26]:
concatenating  = VectorAssembler(inputCols=["age","bmi", "children", "gender_categorical", "smoker_categorical", "region_categorical"],outputCol="features")
results = concatenating.transform(categorised)

In [27]:
for_model = results.select("features", "charges")
for_model.show(truncate=False)

+-----------------------------+-----------+
|features                     |charges    |
+-----------------------------+-----------+
|[19.0,27.9,0.0,1.0,1.0,1.0]  |16884.924  |
|[18.0,33.77,1.0,0.0,0.0,0.0] |1725.5523  |
|[28.0,33.0,3.0,0.0,0.0,0.0]  |4449.462   |
|[33.0,22.705,0.0,0.0,0.0,2.0]|21984.47061|
|[32.0,28.88,0.0,0.0,0.0,2.0] |3866.8552  |
|[31.0,25.74,0.0,1.0,0.0,0.0] |3756.6216  |
|[46.0,33.44,1.0,1.0,0.0,0.0] |8240.5896  |
|[37.0,27.74,3.0,1.0,0.0,2.0] |7281.5056  |
|[37.0,29.83,2.0,0.0,0.0,3.0] |6406.4107  |
|[60.0,25.84,0.0,1.0,0.0,2.0] |28923.13692|
|[25.0,26.22,0.0,0.0,0.0,3.0] |2721.3208  |
|[62.0,26.29,0.0,1.0,1.0,0.0] |27808.7251 |
|[23.0,34.4,0.0,0.0,0.0,1.0]  |1826.843   |
|[56.0,39.82,0.0,1.0,0.0,0.0] |11090.7178 |
|[27.0,42.13,0.0,0.0,1.0,0.0] |39611.7577 |
|[19.0,24.6,1.0,0.0,0.0,1.0]  |1837.237   |
|[52.0,30.78,1.0,1.0,0.0,3.0] |10797.3362 |
|[23.0,23.845,0.0,0.0,0.0,3.0]|2395.17155 |
|[56.0,40.3,0.0,0.0,0.0,1.0]  |10602.385  |
|[30.0,35.3,0.0,0.0,1.0,1.0]  |3

In [29]:
train_data, test_data = for_model.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+------------------+
|summary|           charges|
+-------+------------------+
|  count|               956|
|   mean|13354.916212594144|
| stddev|12200.422155005328|
|    min|         1121.8739|
|    max|       63770.42801|
+-------+------------------+



In [31]:
from pyspark.ml.regression import LinearRegression

lr_model = LinearRegression(featuresCol= "features",labelCol="charges")
training_model = lr_model.fit(train_data)

In [41]:
output = training_model.evaluate(train_data)
print(output.r2)
print(output.meanSquaredError)

0.7658313783481423
34819609.43749611


In [46]:
training_model.write().overwrite().save("file:/home/hduser/lr_model")

In [40]:
result = training_model.evaluate(test_data)
result.predictions.show()

+--------------------+-----------+-------------------+
|            features|    charges|         prediction|
+--------------------+-----------+-------------------+
|(6,[0,1],[18.0,37...|  1141.4451| 3922.8801595123205|
|(6,[0,1],[18.0,43...|  1149.3959|  5815.462486855764|
|(6,[0,1],[18.0,53...|  1163.4627|  9163.877373694157|
|(6,[0,1],[20.0,33...|  1391.5287|  3137.380752403329|
|(6,[0,1],[21.0,35...|  1532.4697|  4127.671980369038|
|(6,[0,1],[21.0,36...|  1534.3045| 4564.4217482175245|
|(6,[0,1],[22.0,26...|  1664.9996| 1514.7776235847396|
|(6,[0,1],[23.0,32...|  1824.2854| 3669.7348991464114|
|(6,[0,1],[23.0,41...|  1837.2819|  6763.379088073185|
|(6,[0,1],[24.0,32...|  1981.5819|  3750.130777427772|
|(6,[0,1],[24.0,35...|  1986.9334|  5023.984266985857|
|(6,[0,1],[33.0,30...|  3704.3545|  5529.172287593878|
|(6,[0,1],[40.0,41...|  5438.7491| 11150.961579808381|
|(6,[0,1],[56.0,34...| 10594.2257| 12946.837028133426|
|(6,[0,1],[60.0,25...| 12142.5786|  11121.06751600382|
|(6,[0,1],